In [1]:
import pandas as pd
import random

In [2]:
# Veut dire que c’est un passage par adresse des 3 variables IX, IY et IZ.
# 3 variables entières qu’il faut initialiser avant le 1er appel de la fonction.
# Veut dire que la fonction est typée, elle retourne un nombre réel.
def alea(IX, IY, IZ):
    # Déclaration d’une variable inter de type réel
    inter = 0.0
    # A chaque exécution de la fonction, les valeurs des 3 variables sont recalculées, d’où un passage par adresse.
    # % : division entière
    # // : reste de la division entière
    IX = 171 * (IX % 177) - 2 * (IX // 177)
    IY = 172 * (IY % 176) - 35 * (IY // 176)
    IZ = 170 * (IZ % 178) - 63 * (IZ // 178)
    # S’il y a un dépassement de capacité sur une variable, on ajoute un nombre très grand, pour la machine 16 bits,
    # pour ramener à une valeur positive. Ce qui marche sur une machine 16 bits marche sur une 32 ou 64 bits.
    if IX < 0:
        IX = IX + 30269
    if IY < 0:
        IY = IY + 30307
    if IZ < 0:
        IZ = IZ + 30323
    inter = ((IX / 30269) + (IY / 30307) + (IZ / 30323))
    return [round(inter - int(inter), 4),IX, IY, IZ]

In [3]:
salaire = [(40000, 30000), (30000, 20000), (20000, 15000), (15000, 10000), (10000, 7500), (7500, 5000), (5000, 3000)]
frequenceSalaire = [5, 10, 10, 15, 20, 20, 20]
age = [(62, 53), (52, 41), (40, 31), (30, 21)]
frequenceAge = [30, 30, 20, 20]
ageEmbauche = [(21, 22), (23, 24), (25, 28), (29, 32), (33, 35), (36, 40)]
frequenceAgeEmbauche = [5, 20, 25, 25, 20, 5]
salaire_embauche=[(32000,24000),(24000,16000),(16000,12000),(12000,8000),(8000,6000),(6000,4000),(4000,3000)]
frequenceSalaireEmbauche=[5,5,10,20,20,20,20]
years = [24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
totEmp=10000
totRet=1000

In [4]:
ensemble_data_frames = {}
for j in range(1, 41):
    data_frames = {}
    for i in range(23, 34) :
        data_frames['Employe_20'+str(i)] = pd.DataFrame(columns=["id", "age", "salaire"])
        data_frames['Retraite_20'+str(i)] = pd.DataFrame(columns=["id", "age", "salaire"])
        data_frames['newRecru_20'+str(i)] = pd.DataFrame(columns=["id", "age", "salaire"])
    ensemble_data_frames['Simulation_'+str(j)] = data_frames

In [5]:
def cumuleFrequenceIntervale(frequences):
    cumules = []
    index = 0
    for i in frequences:
        cumules.append([index, i+index])
        index += i
    return cumules

In [6]:
def indexFromTable(cumules, aleaNumber):
    i = 0
    for intervale in cumules :
        if intervale[1] > aleaNumber*100 :
            return i
        i+=1
    return i

In [7]:
def createEmployeRetraite(simulation, year, IX, IY, IZ):
    dfEmploye = ensemble_data_frames["Simulation_"+str(simulation)]["Employe_20"+str(year)]
    dfRetraite = ensemble_data_frames["Simulation_"+str(simulation)]["Retraite_20"+str(year)]
    for index in range(11000):
        if index >=1:
            IX = alea(IX, IY, IZ)[1]
            IY = alea(IX, IY, IZ)[2]
            IZ = alea(IX, IY, IZ)[3]
        salaireIndex = indexFromTable(cumuleFrequenceIntervale(frequenceSalaire), alea(IX, IY, IZ)[0])
        ageIndex = indexFromTable(cumuleFrequenceIntervale(frequenceAge), alea(IX, IY, IZ)[0])
        i = index+1
        s = random.randrange(salaire[salaireIndex][1], salaire[salaireIndex][0], 100)
        if index <10000:
            a = random.randrange(age[ageIndex][1], age[ageIndex][0], 1)
            new_row = pd.DataFrame( {"id" : [i], "age": [a], "salaire": [s]})
            dfEmploye = pd.concat([dfEmploye, new_row], ignore_index=True)
        else:
            a = random.randint(66, 85)
            new_row = pd.DataFrame( {"id" : [i], "age": [a], "salaire": [s]})
            dfRetraite = pd.concat([dfRetraite, new_row], ignore_index=True)
    ensemble_data_frames["Simulation_"+str(simulation)]["Employe_20"+str(year)] = dfEmploye
    ensemble_data_frames["Simulation_"+str(simulation)]["Retraite_20"+str(year)] = dfRetraite

In [8]:
def pensionnaireFive(dfRetrait) :
    pensionnaire65 = pd.DataFrame(columns=["id", "age", "salaire", "pension"])
    for index, row in dfRetrait.iterrows():
        a = row["age"]
        s = row["salaire"]
        p = ((44*2)/100)*row["salaire"]
        new_row = pd.DataFrame( {"id" : row["id"], "age": [a], "salaire": [s], "pension": [p]})
        pensionnaire65 = pd.concat([pensionnaire65, new_row], ignore_index=True)
    return pensionnaire65

In [11]:
pensionnaireFive(ensemble_data_frames["Simulation_1"]["Retraite_2023"]).head()

In [12]:
def pensionnaireTwo(dfEmploye, dfRetraite):
    pensionnaire62 = pd.DataFrame(columns=["id", "age", "salaire", "pension"])
    pensionnaire65 = pensionnaireFive(dfRetraite)
    pensionnaire62 = pd.concat([pensionnaire62, pensionnaire65], ignore_index=True)
    for index, row in dfEmploye.iterrows():
        if row["age"]>62:
            a = row["age"]
            s = row["salaire"]
            p = ((44*2)/100)*row["salaire"]
            new_row = pd.DataFrame( {"id" : row["id"], "age": [a], "salaire": [s], "pension": [p]})
            pensionnaire62 = pd.concat([pensionnaire62, new_row], ignore_index=True)
    return pensionnaire62

In [13]:
pensionnaireTwo(ensemble_data_frames["Simulation_1"]["Employe_2023"], ensemble_data_frames["Simulation_1"]["Retraite_2023"]).tail()

In [14]:
def newRecrutement(IX, IY, IZ):
    Nouveux_Recrus = pd.DataFrame(columns=["id", "age", "salaire"])
    a=alea(IX, IY, IZ)[0]
    n = ((400 - 250) * a) + 250
    for index in range(int(n)):
        if index >= 0 :
            IX = alea(IX, IY, IZ)[1]
            IY = alea(IX, IY, IZ)[2]
            IZ = alea(IX, IY, IZ)[3]
        salaireIndex = indexFromTable(cumuleFrequenceIntervale(frequenceSalaireEmbauche), alea(IX, IY, IZ)[0])
        ageIndex = indexFromTable(cumuleFrequenceIntervale(frequenceAgeEmbauche), alea(IX, IY, IZ)[0])
        i = index+1
        s = random.randrange(salaire_embauche[salaireIndex][1], salaire_embauche[salaireIndex][0], 100)
        a = random.randrange(ageEmbauche[ageIndex][0], ageEmbauche[ageIndex][1], 1)
        new_row = pd.DataFrame( {"id" : [i], "age": [a], "salaire": [s]})
        Nouveux_Recrus = pd.concat([Nouveux_Recrus, new_row], ignore_index=True)
    return Nouveux_Recrus

In [19]:
def calculEmployeRetraite(simulation, i, IX, IY, IZ):
    EmployeBeta = ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(i-1)]
    RetraiteBeta = ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(i-1)]
    EmployeAlpha = ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(i)]
    RetraiteAlpha = ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(i)]
    for index, row in EmployeBeta.iterrows():
        if row["age"] < 65:
            if i == 25 or i == 30:
                s = row["salaire"]*1.05
            else :
                s = row["salaire"]
            a = row["age"]+1
            new_row = pd.DataFrame({"id" : row["id"], "age": [a], "salaire": [s]})
            EmployeAlpha = pd.concat([EmployeAlpha, new_row], ignore_index=True)
        else :
            if i == 25 or i == 30:
                s = row["salaire"]*1.05
            else :
                s = row["salaire"]
            a = row["age"]+1
            new_row = pd.DataFrame({"id" : row["id"], "age": [a], "salaire": [s]})
            RetraiteAlpha = pd.concat([RetraiteAlpha, new_row], ignore_index=True)
    for index, row in RetraiteBeta.iterrows():
        new_row = pd.DataFrame({"id" : [row["id"]], "age": [row["age"]+1], "salaire": [row["salaire"]]})
        RetraiteAlpha = pd.concat([RetraiteAlpha, new_row], ignore_index=True)
    newRecru = newRecrutement(IX, IY, IZ)
    for index, row in newRecru.iterrows():
        new_row = pd.DataFrame({"id" : [row["id"]], "age": [row["age"]], "salaire": [row["salaire"]]})
        EmployeAlpha = pd.concat([EmployeAlpha, new_row], ignore_index=True)
    ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(i)] = EmployeAlpha
    ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(i)] = RetraiteAlpha
    ensemble_data_frames["Simulation_"+str(simulation)]['newRecru_20'+str(i)] = newRecru

,id,age,salaire
0,1,46,6504.75
1,2,60,10032.75
2,3,63,32193.00
3,4,37,4851.00
4,5,32,3638.25
...,...,...,...
11546,349,29,7200.00
11547,350,31,4700.00
11548,351,33,4800.00
11549,352,23,20200.00


In [20]:
IX = 100
IY = 100
IZ = 100
createEmployeRetraite(1, 23, IX, IY, IZ)
for year in years:
    calculEmployeRetraite(1, year, IX, IY, IZ)

,id,age,salaire
0,108,66,26129.25
1,116,66,23152.50
2,141,66,17529.75
3,197,66,18742.50
4,225,66,11245.50
...,...,...,...
2974,10996,86,9800.00
2975,10997,83,32600.00
2976,10998,93,3000.00
2977,10999,84,9000.00


In [17]:
simulation = 1
year = 24
ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(i)]

,id,age,salaire
0,1,31,7100
1,2,23,12000
2,3,36,3000
3,4,34,3100
4,5,33,3700
...,...,...,...
348,349,29,7200
349,350,31,4700
350,351,33,4800
351,352,23,20200


In [ ]:
ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(i)]

In [ ]:
ensemble_data_frames["Simulation_"+str(simulation)]['newRecru_20'+str(i)]